# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846853974101                   -0.70    4.8         
  2   -7.852323895511       -2.26       -1.53    1.0   40.1ms
  3   -7.852617503618       -3.53       -2.55    1.5   38.0ms
  4   -7.852645957836       -4.55       -2.89    2.2   46.2ms
  5   -7.852646519166       -6.25       -3.22    1.0   34.8ms
  6   -7.852646679695       -6.79       -4.13    1.2   34.1ms
  7   -7.852646686687       -8.16       -5.10    2.2   45.9ms
  8   -7.852646686727      -10.39       -5.62    2.0   39.9ms
  9   -7.852646686727      -13.06       -5.53    1.5   37.1ms
 10   -7.852646686730      -11.60       -6.91    1.0   34.9ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846711397769                   -0.70           4.2         
  2   -7.852526637548       -2.24       -1.64   0.80    2.5    326ms
  3   -7.852636533799       -3.96       -2.72   0.80    1.0   31.2ms
  4   -7.852646545787       -5.00       -3.26   0.80    2.0   43.3ms
  5   -7.852646673794       -6.89       -4.14   0.80    1.0   34.7ms
  6   -7.852646686360       -7.90       -4.79   0.80    1.8   37.9ms
  7   -7.852646686718       -9.45       -5.61   0.80    2.0   38.0ms
  8   -7.852646686729      -10.96       -6.83   0.80    2.0   39.6ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.366576e+01     3.020628e+00
 * time: 0.6248180866241455
     1     1.136376e+00     1.844366e+00
 * time: 0.8868310451507568
     2    -1.613010e+00     2.252216e+00
 * time: 0.9223389625549316
     3    -3.840386e+00     1.954077e+00
 * time: 0.969506025314331
     4    -5.185394e+00     1.862168e+00
 * time: 1.0179400444030762
     5    -6.725505e+00     1.364270e+00
 * time: 1.0655629634857178
     6    -6.748880e+00     2.257601e+00
 * time: 1.0989329814910889
     7    -7.246165e+00     1.855803e+00
 * time: 1.1347410678863525
     8    -7.523329e+00     1.277829e+00
 * time: 1.1835119724273682
     9    -7.688837e+00     5.481212e-01
 * time: 1.2309679985046387
    10    -7.764544e+00     2.045532e-01
 * time: 1.2650270462036133
    11    -7.802449e+00     1.497666e-01
 * time: 1.2985739707946777
    12    -7.832129e+00     5.712558e-02
 * time: 1.3312301635742188
    13    -7.842866e+00     3.775967e-02
 * time: 1.36549496650

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846795524283                   -0.70    4.8         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645877719                   -1.64         
  2   -7.852646686730       -6.09       -3.70    2.24s
  3   -7.852646686730      -13.25       -7.23    248ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 2.533555393163064e-7
|ρ_newton - ρ_scfv| = 4.98266542279815e-7
|ρ_newton - ρ_dm|   = 3.7719791237558444e-10
